## 📦 Installation des dépendances

In [1]:
# Installation des packages nécessaires
!pip install sentence-transformers faiss-cpu pandas numpy scikit-learn umap-learn plotly streamlit fastapi uvicorn

'pip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


## 📚 Imports

In [1]:
import pandas as pd
import numpy as np
import faiss
import pickle
import time
from pathlib import Path
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Sentence Transformers pour les embeddings
from sentence_transformers import SentenceTransformer, CrossEncoder

# Pour la visualisation
import plotly.express as px
import plotly.graph_objects as go
from umap import UMAP

# Pour l'évaluation
from sklearn.metrics.pairwise import cosine_similarity

print("✅ Tous les imports réussis !")

✅ Tous les imports réussis !


# Étape 1 : Construction du Corpus
## 📄 Chargement et nettoyage des données

In [2]:
# Définir le chemin vers les données
data_dir = Path("BD quest_resp medecine")

# Liste des fichiers CSV médicaux
medical_files = [
    "CancerQA.csv",
    "Diabetes_and_Digestive_and_Kidney_DiseasesQA.csv",
    "Disease_Control_and_PreventionQA.csv",
    "Genetic_and_Rare_DiseasesQA.csv",
    "Heart_Lung_and_BloodQA.csv",
    "MedicalQuestionAnswering.csv",
    "Neurological_Disorders_and_StrokeQA.csv",
    "SeniorHealthQA.csv"
]

# Charger tous les fichiers
all_data = []
for file in medical_files:
    try:
        df = pd.read_csv(data_dir / file, encoding='utf-8')
        df['source_file'] = file.replace('.csv', '')
        all_data.append(df)
        print(f"✅ {file}: {len(df)} lignes chargées")
    except Exception as e:
        print(f"❌ Erreur avec {file}: {e}")

# Concaténer tous les DataFrames
corpus_df = pd.concat(all_data, ignore_index=True)
print(f"\n📊 Corpus total: {len(corpus_df)} documents")

✅ CancerQA.csv: 729 lignes chargées
✅ Diabetes_and_Digestive_and_Kidney_DiseasesQA.csv: 1192 lignes chargées
✅ Disease_Control_and_PreventionQA.csv: 270 lignes chargées
✅ Genetic_and_Rare_DiseasesQA.csv: 5388 lignes chargées
✅ Heart_Lung_and_BloodQA.csv: 559 lignes chargées
✅ MedicalQuestionAnswering.csv: 16406 lignes chargées
✅ Neurological_Disorders_and_StrokeQA.csv: 1088 lignes chargées
✅ SeniorHealthQA.csv: 769 lignes chargées

📊 Corpus total: 26401 documents


In [3]:
# Examiner la structure des données
print("📋 Colonnes disponibles:")
print(corpus_df.columns.tolist())
print("\n📊 Aperçu des données:")
corpus_df.head(3)

📋 Colonnes disponibles:
['Question', 'Answer', 'topic', 'split', 'source_file']

📊 Aperçu des données:


,Question,Answer,topic,split,source_file
0,What is (are) Non-Small Cell Lung Cancer ?,Key Points\n - Non-small ce...,cancer,train,CancerQA
1,Who is at risk for Non-Small Cell Lung Cancer? ?,Smoking is the major risk factor for non-small...,cancer,train,CancerQA
2,What are the symptoms of Non-Small Cell Lung C...,Signs of non-small cell lung cancer include a ...,cancer,test,CancerQA


In [4]:
# Nettoyage des données
def clean_text(text):
    """Nettoie le texte en supprimant les caractères indésirables"""
    if pd.isna(text):
        return ""
    
    text = str(text)
    # Supprimer les retours à la ligne multiples
    text = ' '.join(text.split())
    # Supprimer les caractères spéciaux excessifs
    text = text.replace('\n', ' ').replace('\r', ' ')
    return text.strip()

# Nettoyer les colonnes
corpus_df['Question'] = corpus_df['Question'].apply(clean_text)
corpus_df['Answer'] = corpus_df['Answer'].apply(clean_text)

# Supprimer les lignes vides
corpus_df = corpus_df[(corpus_df['Question'] != '') & (corpus_df['Answer'] != '')]

# Créer une colonne de texte combiné pour la recherche
corpus_df['text'] = corpus_df['Question'] + " " + corpus_df['Answer']

# Créer un ID unique pour chaque document
corpus_df['doc_id'] = range(len(corpus_df))

print(f"✅ Nettoyage terminé. {len(corpus_df)} documents valides")

✅ Nettoyage terminé. 26401 documents valides


In [5]:
# Limiter à environ 1500 documents pour le projet (comme demandé: 500-2000)
if len(corpus_df) > 1500:
    corpus_df = corpus_df.sample(n=1500, random_state=42).reset_index(drop=True)
    corpus_df['doc_id'] = range(len(corpus_df))
    print(f"📊 Corpus réduit à {len(corpus_df)} documents")

# Statistiques du corpus
print("\n📊 Statistiques du corpus:")
print(f"Nombre de documents: {len(corpus_df)}")
print(f"Sources: {corpus_df['source_file'].nunique()}")
print(f"\nDistribution par source:")
print(corpus_df['source_file'].value_counts())

📊 Corpus réduit à 1500 documents

📊 Statistiques du corpus:
Nombre de documents: 1500
Sources: 8

Distribution par source:
source_file
MedicalQuestionAnswering                        922
Genetic_and_Rare_DiseasesQA                     324
Neurological_Disorders_and_StrokeQA              69
Diabetes_and_Digestive_and_Kidney_DiseasesQA     61
SeniorHealthQA                                   38
Heart_Lung_and_BloodQA                           36
CancerQA                                         34
Disease_Control_and_PreventionQA                 16
Name: count, dtype: int64


In [6]:
# Sauvegarder le corpus nettoyé
corpus_df.to_csv('docs_medical.csv', index=False)
print("✅ Corpus sauvegardé dans 'docs_medical.csv'")

✅ Corpus sauvegardé dans 'docs_medical.csv'


# Étape 2 : Vectorisation et Indexation
## 🧠 Génération des embeddings avec Sentence-Transformers

In [7]:
# Charger le modèle d'embedding
print("📥 Chargement du modèle Sentence-Transformer...")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print(f"✅ Modèle chargé. Dimension des embeddings: {model.get_sentence_embedding_dimension()}")

📥 Chargement du modèle Sentence-Transformer...
✅ Modèle chargé. Dimension des embeddings: 384


In [8]:
# Générer les embeddings
print("🔄 Génération des embeddings... (cela peut prendre quelques minutes)")
start_time = time.time()

# Encoder les textes
embeddings = model.encode(
    corpus_df['text'].tolist(),
    show_progress_bar=True,
    batch_size=32,
    normalize_embeddings=True  # Normaliser pour utiliser la similarité cosinus
)

elapsed_time = time.time() - start_time
print(f"\n✅ Embeddings générés en {elapsed_time:.2f}s")
print(f"📊 Shape des embeddings: {embeddings.shape}")

🔄 Génération des embeddings... (cela peut prendre quelques minutes)


Batches: 100%|██████████| 47/47 [01:02<00:00,  1.34s/it]


✅ Embeddings générés en 62.92s
📊 Shape des embeddings: (1500, 384)


In [9]:
# Sauvegarder les embeddings
np.save('embeddings_medical.npy', embeddings)
print("✅ Embeddings sauvegardés dans 'embeddings_medical.npy'")

✅ Embeddings sauvegardés dans 'embeddings_medical.npy'


## Test du modèle - Recherche sémantique

In [25]:
# Test de recherche avec le modèle
def test_semantic_search(query, model, embeddings, corpus_df, top_k=5):
    """Test de recherche sémantique avec le vrai modèle"""
    print(f"\n🔍 Question: {query}")
    print("=" * 80)
    
    # Encoder la requête avec le modèle
    query_embedding = model.encode([query], normalize_embeddings=True)
    
    # Calculer les similarités
    similarities = np.dot(embeddings, query_embedding.T).flatten()
    
    # Trier par score
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    # Afficher les résultats
    for rank, idx in enumerate(top_indices, 1):
        doc = corpus_df.iloc[idx]
        score = similarities[idx]
        print(f"\n📄 Résultat #{rank} | Score: {score:.4f}")
        print(f"Question: {doc['Question'][:100]}...")
        print(f"Réponse: {doc['Answer'][:150]}...")
        print(f"Source: {doc['source_file']}")
    
    return top_indices, similarities[top_indices]

# Tester avec plusieurs questions médicales
test_questions = [
    "What are the symptoms of diabetes?",
    "How to treat cancer?",
    "What causes heart disease?",
    "What are neurological disorders?"
]

print("🧪 Test du modèle de recherche sémantique")
print("=" * 80)

for query in test_questions:
    indices, scores = test_semantic_search(query, model, embeddings, corpus_df, top_k=3)
    print("\n" + "-" * 80)

🧪 Test du modèle de recherche sémantique

🔍 Question: What are the symptoms of diabetes?

📄 Résultat #1 | Score: 0.7964
Question: What are the symptoms of Diabetes ?...
Réponse: Many people with diabetes experience one or more symptoms, including extreme thirst or hunger, a frequent need to urinate and/or fatigue. Some lose we...
Source: MedicalQuestionAnswering

📄 Résultat #2 | Score: 0.7698
Question: What are the symptoms of Your Guide to Diabetes: Type 1 and Type 2 ?...
Réponse: The signs and symptoms of diabetes are - being very thirsty - urinating often - feeling very hungry - feeling very tired - losing weight without tryin...
Source: Diabetes_and_Digestive_and_Kidney_DiseasesQA

📄 Résultat #3 | Score: 0.6423
Question: What are the symptoms of Prevent diabetes problems: Keep your kidneys healthy ?...
Réponse: In the early stages, diabetic kidney disease does not have any symptoms. Kidney disease happens so slowly that you may not feel sick at all for many y...
Source: MedicalQues

## 🗂️ Création de l'index FAISS

In [10]:
# Créer l'index FAISS
dimension = embeddings.shape[1]

# Option 1: IndexFlatIP (Inner Product) - Pour les embeddings normalisés
index = faiss.IndexFlatIP(dimension)

# Ajouter les embeddings à l'index
index.add(embeddings.astype('float32'))

print(f"✅ Index FAISS créé")
print(f"📊 Nombre de vecteurs dans l'index: {index.ntotal}")

✅ Index FAISS créé
📊 Nombre de vecteurs dans l'index: 1500


In [11]:
# Sauvegarder l'index FAISS
faiss.write_index(index, 'medical_faiss.index')
print("✅ Index FAISS sauvegardé dans 'medical_faiss.index'")

✅ Index FAISS sauvegardé dans 'medical_faiss.index'


## 🔍 Fonction de recherche

In [12]:
def search_documents(query: str, top_k: int = 10) -> List[Dict]:
    """
    Recherche les documents les plus pertinents pour une requête
    
    Args:
        query: La requête en langage naturel
        top_k: Nombre de résultats à retourner
    
    Returns:
        Liste de dictionnaires contenant les documents pertinents
    """
    # Encoder la requête
    query_embedding = model.encode([query], normalize_embeddings=True)
    
    # Rechercher dans l'index FAISS
    scores, indices = index.search(query_embedding.astype('float32'), top_k)
    
    # Préparer les résultats
    results = []
    for i, (idx, score) in enumerate(zip(indices[0], scores[0])):
        doc = corpus_df.iloc[idx]
        results.append({
            'rank': i + 1,
            'doc_id': int(doc['doc_id']),
            'question': doc['Question'],
            'answer': doc['Answer'],
            'source': doc['source_file'],
            'score': float(score)
        })
    
    return results

print("✅ Fonction de recherche créée")

✅ Fonction de recherche créée


In [13]:
# Test de la recherche
test_query = "What are the symptoms of diabetes?"
print(f"🔍 Requête de test: {test_query}\n")

results = search_documents(test_query, top_k=5)

for result in results:
    print(f"Rang {result['rank']} | Score: {result['score']:.4f}")
    print(f"Question: {result['question'][:100]}...")
    print(f"Réponse: {result['answer'][:200]}...")
    print(f"Source: {result['source']}")
    print("-" * 80)

🔍 Requête de test: What are the symptoms of diabetes?

Rang 1 | Score: 0.7964
Question: What are the symptoms of Diabetes ?...
Réponse: Many people with diabetes experience one or more symptoms, including extreme thirst or hunger, a frequent need to urinate and/or fatigue. Some lose weight without trying. Additional signs include sore...
Source: MedicalQuestionAnswering
--------------------------------------------------------------------------------
Rang 2 | Score: 0.7699
Question: What are the symptoms of Your Guide to Diabetes: Type 1 and Type 2 ?...
Réponse: The signs and symptoms of diabetes are - being very thirsty - urinating often - feeling very hungry - feeling very tired - losing weight without trying - sores that heal slowly - dry, itchy skin - fee...
Source: Diabetes_and_Digestive_and_Kidney_DiseasesQA
--------------------------------------------------------------------------------
Rang 3 | Score: 0.6423
Question: What are the symptoms of Prevent diabetes problems: Keep your

## 🎯 Re-ranking avec CrossEncoder

In [14]:
# Charger le modèle de re-ranking
print("📥 Chargement du CrossEncoder pour le re-ranking...")
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("✅ CrossEncoder chargé")

📥 Chargement du CrossEncoder pour le re-ranking...
✅ CrossEncoder chargé


In [15]:
def search_with_reranking(query: str, top_k: int = 10, rerank_top_n: int = 50) -> List[Dict]:
    """
    Recherche avec re-ranking pour améliorer la pertinence
    
    Args:
        query: La requête en langage naturel
        top_k: Nombre de résultats finaux
        rerank_top_n: Nombre de candidats à re-ranker
    
    Returns:
        Liste de résultats re-rankés
    """
    # Étape 1: Récupération initiale avec FAISS
    initial_results = search_documents(query, top_k=rerank_top_n)
    
    # Étape 2: Re-ranking avec CrossEncoder
    pairs = [[query, result['answer']] for result in initial_results]
    cross_scores = cross_encoder.predict(pairs)
    
    # Étape 3: Trier par score du CrossEncoder
    for result, cross_score in zip(initial_results, cross_scores):
        result['cross_score'] = float(cross_score)
    
    # Trier et retourner les top_k meilleurs
    reranked_results = sorted(initial_results, key=lambda x: x['cross_score'], reverse=True)[:top_k]
    
    # Mettre à jour les rangs
    for i, result in enumerate(reranked_results):
        result['rank'] = i + 1
    
    return reranked_results

print("✅ Fonction de recherche avec re-ranking créée")

✅ Fonction de recherche avec re-ranking créée


In [16]:
# Test avec re-ranking
print(f"🔍 Requête avec re-ranking: {test_query}\n")

reranked_results = search_with_reranking(test_query, top_k=5)

for result in reranked_results:
    print(f"Rang {result['rank']} | FAISS Score: {result['score']:.4f} | Cross Score: {result['cross_score']:.4f}")
    print(f"Question: {result['question'][:100]}...")
    print(f"Source: {result['source']}")
    print("-" * 80)

🔍 Requête avec re-ranking: What are the symptoms of diabetes?

Rang 1 | FAISS Score: 0.7699 | Cross Score: 9.2198
Question: What are the symptoms of Your Guide to Diabetes: Type 1 and Type 2 ?...
Source: Diabetes_and_Digestive_and_Kidney_DiseasesQA
--------------------------------------------------------------------------------
Rang 2 | FAISS Score: 0.7964 | Cross Score: 8.8553
Question: What are the symptoms of Diabetes ?...
Source: MedicalQuestionAnswering
--------------------------------------------------------------------------------
Rang 3 | FAISS Score: 0.5716 | Cross Score: 4.0853
Question: What are the symptoms of Diabetic mastopathy ?...
Source: MedicalQuestionAnswering
--------------------------------------------------------------------------------
Rang 4 | FAISS Score: 0.6423 | Cross Score: 2.4375
Question: What are the symptoms of Prevent diabetes problems: Keep your kidneys healthy ?...
Source: MedicalQuestionAnswering
------------------------------------------------------

# Étape 3 : API Backend avec FastAPI
## 🚀 Création de l'API REST

In [17]:
# Créer le fichier de l'API
api_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, CrossEncoder
import uvicorn

# Initialiser FastAPI
app = FastAPI(title="Medical Q&A Search API", version="1.0.0")

# Charger les ressources
print("Loading resources...")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
index = faiss.read_index('medical_faiss.index')
corpus_df = pd.read_csv('docs_medical.csv')
print(f"Resources loaded. Index contains {index.ntotal} documents")

# Modèles Pydantic
class QueryRequest(BaseModel):
    query: str
    top_k: int = 10
    use_reranking: bool = True

class SearchResult(BaseModel):
    rank: int
    doc_id: int
    question: str
    answer: str
    source: str
    score: float
    cross_score: Optional[float] = None

class QueryResponse(BaseModel):
    query: str
    results: List[SearchResult]
    total_results: int
    processing_time: float

# Endpoints
@app.get("/")
def root():
    return {
        "message": "Medical Q&A Search API",
        "version": "1.0.0",
        "endpoints": ["/query", "/docs/{doc_id}", "/stats"]
    }

@app.post("/query", response_model=QueryResponse)
def query_documents(request: QueryRequest):
    """Rechercher des documents pertinents"""
    import time
    start_time = time.time()
    
    try:
        # Encoder la requête
        query_embedding = model.encode([request.query], normalize_embeddings=True)
        
        # Recherche FAISS
        top_k = 50 if request.use_reranking else request.top_k
        scores, indices = index.search(query_embedding.astype('float32'), top_k)
        
        # Préparer les résultats
        results = []
        for idx, score in zip(indices[0], scores[0]):
            doc = corpus_df.iloc[idx]
            results.append({
                'doc_id': int(doc['doc_id']),
                'question': doc['Question'],
                'answer': doc['Answer'],
                'source': doc['source_file'],
                'score': float(score)
            })
        
        # Re-ranking si demandé
        if request.use_reranking:
            pairs = [[request.query, r['answer']] for r in results]
            cross_scores = cross_encoder.predict(pairs)
            
            for result, cross_score in zip(results, cross_scores):
                result['cross_score'] = float(cross_score)
            
            results = sorted(results, key=lambda x: x['cross_score'], reverse=True)[:request.top_k]
        else:
            results = results[:request.top_k]
        
        # Ajouter les rangs
        for i, result in enumerate(results):
            result['rank'] = i + 1
        
        processing_time = time.time() - start_time
        
        return {
            "query": request.query,
            "results": results,
            "total_results": len(results),
            "processing_time": processing_time
        }
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/docs/{doc_id}")
def get_document(doc_id: int):
    """Récupérer un document par son ID"""
    try:
        doc = corpus_df[corpus_df['doc_id'] == doc_id]
        if len(doc) == 0:
            raise HTTPException(status_code=404, detail="Document not found")
        
        doc = doc.iloc[0]
        return {
            "doc_id": int(doc['doc_id']),
            "question": doc['Question'],
            "answer": doc['Answer'],
            "source": doc['source_file'],
            "topic": doc.get('topic', 'N/A')
        }
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/stats")
def get_stats():
    """Statistiques du corpus"""
    return {
        "total_documents": len(corpus_df),
        "index_size": index.ntotal,
        "sources": corpus_df['source_file'].value_counts().to_dict(),
        "model": "all-MiniLM-L6-v2",
        "cross_encoder": "ms-marco-MiniLM-L-6-v2"
    }

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

with open('api_medical.py', 'w', encoding='utf-8') as f:
    f.write(api_code)

print("✅ Fichier API créé: api_medical.py")
print("\nPour lancer l'API, exécutez:")
print("python api_medical.py")
print("ou")
print("uvicorn api_medical:app --reload")

✅ Fichier API créé: api_medical.py

Pour lancer l'API, exécutez:
python api_medical.py
ou
uvicorn api_medical:app --reload


# Étape 4 : Interface Web avec Streamlit
## 🎨 Création de l'interface utilisateur

In [18]:
# Créer le fichier Streamlit
streamlit_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, CrossEncoder
import plotly.express as px
import plotly.graph_objects as go
from umap import UMAP
import time

# Configuration de la page
st.set_page_config(
    page_title="Recherche Médicale Sémantique",
    page_icon="🏥",
    layout="wide"
)

# Style CSS personnalisé
st.markdown("""
    <style>
    .main-header {
        font-size: 3rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }
    .result-card {
        background-color: #f0f2f6;
        padding: 1rem;
        border-radius: 0.5rem;
        margin-bottom: 1rem;
    }
    </style>
""", unsafe_allow_html=True)

# Charger les ressources
@st.cache_resource
def load_resources():
    """Charger tous les modèles et données"""
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    index = faiss.read_index('medical_faiss.index')
    corpus_df = pd.read_csv('docs_medical.csv')
    embeddings = np.load('embeddings_medical.npy')
    return model, cross_encoder, index, corpus_df, embeddings

# Fonction de recherche
def search_with_reranking(query, model, cross_encoder, index, corpus_df, top_k=10):
    """Recherche avec re-ranking"""
    start_time = time.time()
    
    # Encoder la requête
    query_embedding = model.encode([query], normalize_embeddings=True)
    
    # Recherche FAISS
    scores, indices = index.search(query_embedding.astype('float32'), 50)
    
    # Préparer les résultats
    results = []
    for idx, score in zip(indices[0], scores[0]):
        doc = corpus_df.iloc[idx]
        results.append({
            'doc_id': int(doc['doc_id']),
            'question': doc['Question'],
            'answer': doc['Answer'],
            'source': doc['source_file'],
            'score': float(score)
        })
    
    # Re-ranking
    pairs = [[query, r['answer']] for r in results]
    cross_scores = cross_encoder.predict(pairs)
    
    for result, cross_score in zip(results, cross_scores):
        result['cross_score'] = float(cross_score)
    
    results = sorted(results, key=lambda x: x['cross_score'], reverse=True)[:top_k]
    
    for i, result in enumerate(results):
        result['rank'] = i + 1
    
    elapsed = time.time() - start_time
    return results, elapsed

# Interface principale
def main():
    # Titre
    st.markdown('<h1 class="main-header">🏥 Recherche Médicale Sémantique</h1>', unsafe_allow_html=True)
    st.markdown("### Système de recherche intelligent pour questions médicales")
    
    # Charger les ressources
    with st.spinner("Chargement des modèles..."):
        model, cross_encoder, index, corpus_df, embeddings = load_resources()
    
    # Barre latérale
    with st.sidebar:
        st.header("⚙️ Paramètres")
        top_k = st.slider("Nombre de résultats", 1, 20, 10)
        
        st.markdown("---")
        st.header("📊 Statistiques du corpus")
        st.metric("Documents totaux", len(corpus_df))
        st.metric("Vecteurs indexés", index.ntotal)
        
        st.markdown("---")
        st.header("📚 Sources")
        source_counts = corpus_df['source_file'].value_counts()
        for source, count in source_counts.items():
            st.text(f"{source[:20]}...: {count}")
    
    # Zone de recherche
    st.markdown("### 🔍 Entrez votre question médicale")
    query = st.text_input(
        "",
        placeholder="Ex: What are the symptoms of diabetes?",
        label_visibility="collapsed"
    )
    
    # Exemples de requêtes
    st.markdown("**Exemples de questions:**")
    col1, col2, col3 = st.columns(3)
    with col1:
        if st.button("🩺 Symptoms of cancer"):
            query = "What are the symptoms of cancer?"
    with col2:
        if st.button("💊 Treatment for diabetes"):
            query = "What is the treatment for diabetes?"
    with col3:
        if st.button("🧠 Neurological disorders"):
            query = "What are common neurological disorders?"
    
    # Recherche
    if query:
        with st.spinner("Recherche en cours..."):
            results, elapsed = search_with_reranking(
                query, model, cross_encoder, index, corpus_df, top_k
            )
        
        # Afficher les métriques
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("⏱️ Temps de traitement", f"{elapsed:.3f}s")
        with col2:
            st.metric("📄 Résultats trouvés", len(results))
        with col3:
            st.metric("🎯 Score moyen", f"{np.mean([r['cross_score'] for r in results]):.3f}")
        
        st.markdown("---")
        
        # Afficher les résultats
        st.markdown("### 📋 Résultats")
        for result in results:
            with st.expander(
                f"#{result['rank']} - {result['question'][:80]}... (Score: {result['cross_score']:.3f})",
                expanded=(result['rank'] == 1)
            ):
                st.markdown(f"**Question:** {result['question']}")
                st.markdown(f"**Réponse:** {result['answer']}")
                
                col1, col2, col3 = st.columns(3)
                with col1:
                    st.caption(f"📁 Source: {result['source']}")
                with col2:
                    st.caption(f"🔢 Doc ID: {result['doc_id']}")
                with col3:
                    st.caption(f"⭐ FAISS: {result['score']:.3f} | Cross: {result['cross_score']:.3f}")
        
        # Visualisation des scores
        st.markdown("---")
        st.markdown("### 📊 Distribution des scores")
        
        scores_df = pd.DataFrame(results)
        fig = go.Figure()
        fig.add_trace(go.Bar(
            x=scores_df['rank'],
            y=scores_df['score'],
            name='FAISS Score',
            marker_color='lightblue'
        ))
        fig.add_trace(go.Bar(
            x=scores_df['rank'],
            y=scores_df['cross_score'],
            name='Cross-Encoder Score',
            marker_color='orange'
        ))
        fig.update_layout(
            title="Comparaison des scores FAISS vs Cross-Encoder",
            xaxis_title="Rang",
            yaxis_title="Score",
            barmode='group'
        )
        st.plotly_chart(fig, use_container_width=True)

if __name__ == "__main__":
    main()
'''

with open('app_streamlit.py', 'w', encoding='utf-8') as f:
    f.write(streamlit_code)

print("✅ Fichier Streamlit créé: app_streamlit.py")
print("\nPour lancer l'application, exécutez:")
print("streamlit run app_streamlit.py")

✅ Fichier Streamlit créé: app_streamlit.py

Pour lancer l'application, exécutez:
streamlit run app_streamlit.py


# Étape 5 : Évaluation et Visualisation
## 📈 Métriques de performance

In [19]:
# Créer des requêtes de test
test_queries = [
    "What are the symptoms of diabetes?",
    "How is cancer treated?",
    "What causes heart disease?",
    "What are neurological disorders?",
    "How to prevent stroke?",
    "What is genetic testing?",
    "Symptoms of kidney disease",
    "Treatment for blood disorders",
    "What causes rare diseases?",
    "Health issues in seniors"
]

print(f"✅ {len(test_queries)} requêtes de test créées")

✅ 10 requêtes de test créées


In [20]:
# Évaluer les performances
def evaluate_system(queries, top_k=10):
    """Évaluer le système sur un ensemble de requêtes"""
    latencies = []
    all_scores = []
    
    for query in queries:
        start_time = time.time()
        results = search_with_reranking(query, top_k=top_k)
        latency = time.time() - start_time
        
        latencies.append(latency)
        all_scores.extend([r['cross_score'] for r in results])
    
    metrics = {
        'avg_latency': np.mean(latencies),
        'max_latency': np.max(latencies),
        'min_latency': np.min(latencies),
        'avg_score': np.mean(all_scores),
        'total_queries': len(queries)
    }
    
    return metrics, latencies, all_scores

print("🔄 Évaluation du système...")
metrics, latencies, all_scores = evaluate_system(test_queries)

print("\n📊 Métriques de performance:")
print(f"Latence moyenne: {metrics['avg_latency']:.3f}s")
print(f"Latence min: {metrics['min_latency']:.3f}s")
print(f"Latence max: {metrics['max_latency']:.3f}s")
print(f"Score moyen: {metrics['avg_score']:.3f}")
print(f"Requêtes testées: {metrics['total_queries']}")

🔄 Évaluation du système...

📊 Métriques de performance:
Latence moyenne: 5.821s
Latence min: 4.489s
Latence max: 6.756s
Score moyen: 0.250
Requêtes testées: 10


In [21]:
# Visualisation des latences
fig_latency = go.Figure()
fig_latency.add_trace(go.Bar(
    x=[f"Q{i+1}" for i in range(len(latencies))],
    y=latencies,
    marker_color='lightblue',
    name='Latence'
))
fig_latency.add_hline(
    y=metrics['avg_latency'], 
    line_dash="dash", 
    line_color="red",
    annotation_text=f"Moyenne: {metrics['avg_latency']:.3f}s"
)
fig_latency.update_layout(
    title="Latence par requête",
    xaxis_title="Requête",
    yaxis_title="Temps (secondes)",
    showlegend=False
)
fig_latency.show()

## 🎨 Visualisation des embeddings avec UMAP

In [22]:
# Réduire les dimensions avec UMAP
print("🔄 Réduction de dimensionnalité avec UMAP...")
umap_model = UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)

# Échantillonner si trop de données
sample_size = min(500, len(embeddings))
sample_indices = np.random.choice(len(embeddings), sample_size, replace=False)
embeddings_sample = embeddings[sample_indices]
corpus_sample = corpus_df.iloc[sample_indices]

# Réduction UMAP
embeddings_2d = umap_model.fit_transform(embeddings_sample)
print("✅ Réduction UMAP terminée")

🔄 Réduction de dimensionnalité avec UMAP...
✅ Réduction UMAP terminée


In [23]:
# Créer le DataFrame pour la visualisation
viz_df = pd.DataFrame({
    'x': embeddings_2d[:, 0],
    'y': embeddings_2d[:, 1],
    'source': corpus_sample['source_file'].values,
    'question': corpus_sample['Question'].str[:100].values
})

# Visualisation interactive
fig_umap = px.scatter(
    viz_df,
    x='x',
    y='y',
    color='source',
    hover_data=['question'],
    title="Visualisation UMAP des embeddings médicaux",
    labels={'x': 'UMAP 1', 'y': 'UMAP 2'},
    width=1000,
    height=700
)
fig_umap.update_traces(marker=dict(size=8, opacity=0.7))
fig_umap.show()

## 📊 Tableau de métriques

In [24]:
# Créer un tableau récapitulatif
metrics_table = pd.DataFrame({
    'Métrique': [
        'Nombre de documents',
        'Dimension des embeddings',
        'Latence moyenne',
        'Latence minimale',
        'Latence maximale',
        'Score moyen',
        'Modèle d\'embedding',
        'Modèle de re-ranking'
    ],
    'Valeur': [
        f"{len(corpus_df)}",
        f"{embeddings.shape[1]}",
        f"{metrics['avg_latency']:.3f}s",
        f"{metrics['min_latency']:.3f}s",
        f"{metrics['max_latency']:.3f}s",
        f"{metrics['avg_score']:.3f}",
        "all-MiniLM-L6-v2",
        "ms-marco-MiniLM-L-6-v2"
    ]
})

print("\n📊 Tableau récapitulatif des métriques:")
metrics_table


📊 Tableau récapitulatif des métriques:


,Métrique,Valeur
0,Nombre de documents,1500
1,Dimension des embeddings,384
2,Latence moyenne,5.821s
3,Latence minimale,4.489s
4,Latence maximale,6.756s
5,Score moyen,0.250
6,Modèle d'embedding,all-MiniLM-L6-v2
7,Modèle de re-ranking,ms-marco-MiniLM-L-6-v2


# 🎯 Résumé du Projet

## Ce qui a été réalisé:

### ✅ Étape 1: Construction du corpus
- Collecte de ~1500 documents médicaux
- Nettoyage et préparation des données
- Sauvegarde dans `docs_medical.csv`

### ✅ Étape 2: Vectorisation et Indexation
- Génération des embeddings avec `all-MiniLM-L6-v2`
- Création de l'index FAISS (IndexFlatIP)
- Sauvegarde de l'index dans `medical_faiss.index`

### ✅ Étape 3: API Backend
- API REST avec FastAPI
- Endpoints: `/query`, `/docs/{id}`, `/stats`
- Re-ranking avec CrossEncoder
- Fichier: `api_medical.py`

### ✅ Étape 4: Interface Web
- Application Streamlit complète
- Interface intuitive et responsive
- Visualisations interactives
- Fichier: `app_streamlit.py`

### ✅ Étape 5: Évaluation et Visualisation
- Calcul des métriques de performance
- Visualisation UMAP des embeddings
- Graphiques de latence et scores

## 🚀 Comment utiliser le système:

### 1. Lancer l'API (optionnel):
```bash
python api_medical.py
```

### 2. Lancer l'application Streamlit:
```bash
streamlit run app_streamlit.py
```

### 3. Utiliser directement dans le notebook:
```python
results = search_with_reranking("What are the symptoms of diabetes?", top_k=10)
```

## 📁 Fichiers générés:
- `docs_medical.csv` - Corpus nettoyé
- `embeddings_medical.npy` - Vecteurs d'embeddings
- `medical_faiss.index` - Index FAISS
- `api_medical.py` - API FastAPI
- `app_streamlit.py` - Application Streamlit

## 🎓 Technologies utilisées:
- **Embeddings**: Sentence-Transformers (all-MiniLM-L6-v2)
- **Indexation**: FAISS (IndexFlatIP)
- **Re-ranking**: CrossEncoder (ms-marco-MiniLM-L-6-v2)
- **Backend**: FastAPI
- **Frontend**: Streamlit
- **Visualisation**: Plotly, UMAP

## 🌟 Points forts du projet:
- Système de recherche sémantique performant
- Re-ranking pour améliorer la pertinence
- Interface utilisateur intuitive
- API REST complète et documentée
- Visualisations interactives
- Métriques de performance détaillées